In [75]:
import cv2

In [76]:
import mediapipe as mp

In [89]:
import numpy as np

In [78]:
import os
import glob
from matplotlib import pyplot as plt

In [79]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose



# For static images:
IMAGE_FILES = glob.glob("/home/junkai/Projects/pozz/test_set/bad/*.jpg")
data = np.zeros(( len(IMAGE_FILES),33, 4,))
with mp_pose.Pose(
    static_image_mode=True,
    model_complexity=2,
    enable_segmentation=True,
    min_detection_confidence=0.5) as pose:
    for idx, file in enumerate(IMAGE_FILES):
        image = cv2.imread(file)
        image_height, image_width, _ = image.shape
        # Convert the BGR image to RGB before processing.
        results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        pose_landmarks = results.pose_landmarks

    

        for i, data_point in enumerate(pose_landmarks.landmark):
            if data_point.visibility >= 0.5:
                data[idx][i][0] =  data_point.x
                data[idx][i][1] =  data_point.y
                data[idx][i][2] =  data_point.z
                data[idx][i][3] =  data_point.visibility

    

In [80]:
bad = data.reshape(len(IMAGE_FILES), -1)

In [81]:
# For static images:
IMAGE_FILES = glob.glob("/home/junkai/Projects/pozz/test_set/good/*.jpg")
data = np.zeros(( len(IMAGE_FILES),33, 4,))
with mp_pose.Pose(
    static_image_mode=True,
    model_complexity=2,
    enable_segmentation=True,
    min_detection_confidence=0.5) as pose:
    for idx, file in enumerate(IMAGE_FILES):
        image = cv2.imread(file)
        image_height, image_width, _ = image.shape
        # Convert the BGR image to RGB before processing.
        results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        pose_landmarks = results.pose_landmarks

    

        for i, data_point in enumerate(pose_landmarks.landmark):
            if data_point.visibility >= 0.5:
                data[idx][i][0] =  data_point.x # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image)

    # Draw the pose annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Pose', cv2.flip(image, 1))
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()
                data[idx][i][1] =  data_point.y
                data[idx][i][2] =  data_point.z
                data[idx][i][3] =  data_point.visibility


In [82]:
good = data.reshape(len(IMAGE_FILES), -1)

In [83]:
bad.shape

(329, 132)

In [84]:
data = np.concatenate((good, bad))
data.shape

(493, 132)

In [85]:
good_label = np.zeros((good.shape[0], 2))
good_label[:,0] = np.ones((good.shape[0]))
bad_label = np.zeros((bad.shape[0], 2))
bad_label[:, 1] = np.ones((bad.shape[0]))

In [86]:
labels = np.concatenate((good_label, bad_label))

In [87]:
labels.shape

(493, 2)

In [88]:
np.save("test_datas", data)
np.save("test_labels", labels)